In [ ]:
from cgeniepy.table import ScatterData
import xarray as xr

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["font.family"] = "Helvetica"

x = ScatterData("./LGM_d13c_CLEAN.txt", sep='\t', header=0)
x.data.columns = ['lon', 'lat', 'depth', 'd13C', 'core']
#x.set_index(['lat'])
x.data
## convert data frame to float expce the core column
#import pandas as pd

#x.data = x.data.apply(pd.to_numeric, errors='coerce')
#x.drop_na(subset=['d13C'])
#x.detect_basin()

In [ ]:
isinstance(x.data.index, pd.core.indexes.range.RangeIndex)

In [ ]:
import numpy as np
x = np.random.rand(36, 36)
y = x + np.random.rand(36, 36) * 0.5

## calculate skill score
from cgeniepy.skill import ArrComparison
ms = ArrComparison(x, y)
ms.plot()

In [ ]:
x = ScatterData("/Users/yingrui/Science/lgm_foram_census/tidy/forcens_fg_a_tidy.csv")
x.set_index(['Latitude','Longitude'])

In [ ]:
x.index

In [ ]:
from cgeniepy.model import GenieModel
import matplotlib.pyplot as plt
from cartopy import crs as ccrs

model1 = GenieModel("/Users/yingrui/Science/lgm_foram_niche/model/muffin.CBE.worlg4.BASESFeTDTL.SPIN", gemflag='biogem')
model2 = GenieModel("/Users/yingrui/Downloads/EXP.R07sm_Corg2", gemflag='biogem')

sst1 = model1.get_var("ocn_sur_temp").isel(time=-1)
sst2 = model2.get_var("ocn_sur_temp").isel(time=-1)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), subplot_kw={'projection': ccrs.Robinson(central_longitude=-80)})

## use differnet colormap
sst1.aes_dict['pcolormesh_kwargs']['cmap'] = plt.cm.Spectral_r
sst1.aes_dict['pcolormesh_kwargs']['vmax'] = 35
sst1.aes_dict['pcolormesh_kwargs']['vmin'] = 0

sst2.aes_dict['pcolormesh_kwargs']['cmap'] = plt.cm.Spectral_r
sst2.aes_dict['pcolormesh_kwargs']['vmax'] = 35
sst2.aes_dict['pcolormesh_kwargs']['vmin'] = 0

sst1.aes_dict['colorbar_label_kwargs']['label'] = "Sea surface temperature (°C)"
sst2.aes_dict['colorbar_label_kwargs']['label'] = "Sea surface temperature (°C)"

sst1.plot(ax=ax[0], colorbar=True, outline=True)
sst2.plot(ax=ax[1], colorbar=True, outline=True)

ax[0].set_title("Modern (0 Ma)")
ax[1].set_title("PETM (55 Ma)")

## save 
plt.savefig("fig1.png", dpi=300, bbox_inches="tight")

In [1]:
from cgeniepy.model import GenieModel

pi_model = GenieModel("/Users/yingrui/Science/lgm_foram_niche/model/muffin.CBE.worlg4.BASESFeTDTL.SPIN", gemflag='biogem')

x = pi_model.get_var("ocn_sur_temp").isel(time=-1).to_ScatterData()

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
sst2 = model2.get_var("ocn_sur_temp").isel(time=-1)
sst2.array

In [ ]:
sst1.array.attrs['units'] = '°C'

In [ ]:
from cgeniepy.model import GenieModel

test_model = GenieModel("/Users/yingrui/Science/lgm_bcp/model/muffin.CB.worlg4.BASESFeTDTL.SPIN", gemflag="biogem")
test_array = test_model.get_var("ocn_temp")

## search for the nearest point
point = [0,0,30]

test_array.search_grid(point, ignore_na=True)

In [ ]:
from cgeniepy.model import GenieModel
import cmocean as cmo
import matplotlib.pyplot as plt

model = GenieModel("/Users/yingrui/Science/lgm_foram_niche/model/muffin.CBE.worlg4.BASESFeTDTL.SPIN")

fig, ax = plt.subplots(2,1, figsize=(6, 6))

atlatic_po4 = model.get_var('ocn_PO4').isel(time=-1).mask_basin(base='worjh2',basin='Atlantic', subbasin='').mean(dim='lon')

atlatic_po4.aes_dict['pcolormesh_kwargs']['cmap'] = cmo.cm.thermal
atlatic_po4.aes_dict['contour_kwargs']['levels'] = 12

atlatic_po4.plot(ax=ax[0], colorbar=True, contour=True)
atlatic_po4.interpolate().plot(ax=ax[1], colorbar=True, contour=True)

ax[0].set_title("Original")
ax[1].set_title("Interpolated")

fig.tight_layout()
plt.savefig("../publication/fig2.png", dpi=300, bbox_inches="tight")

In [ ]:
from cgeniepy.model import GenieModel
import numpy as np

pi_model = GenieModel("/Users/yingrui/Science/lgm_bcp/model/muffin.CB.worlg4.BASESFeTDTL.SPIN", gemflag='biogem')

o2 = pi_model.get_var('ocn_temp').isel(time=-1) ##mol/kg
ocn_vol = pi_model.grid_volume().isel(time=-1) ##m3

print("average of o2 weighted by ocean grid volume", o2.weighted_average(ocn_vol.array.values))

## unweighted average of o2
print("unweighted average of o2", o2.array.mean().values)

In [ ]:
from cgeniepy.model import GenieModel

pi_model = GenieModel("/Users/yingrui/Science/lgm_foram_niche/model/muffin.CBE.worlg4.BASESFeTDTL.SPIN", gemflag='biogem')

# Example usage
filename = "biogem_year_09999_500_diag_GLOBAL_AVERAGE.res"
pi_model.get_diag_avg(filename)

In [ ]:
from cgeniepy.model import GenieModel

pi_model = GenieModel("/Users/yingrui/Science/lgm_foram_niche/model/muffin.CBE.worlg4.BASESFeTDTL.historical", gemflag='biogem')

df = pi_model.get_ts("ocn_PO4")

In [ ]:
# Set time as index
df.set_index('time (yr)', inplace=True)

# Get columns to plot (exclude time and model)
plot_cols = [col for col in df.columns if col not in ['time (yr)', 'model']]

# Create a figure and a grid of subplots
num_rows = 3  # Number of rows in the grid
num_cols = (len(plot_cols) + num_rows - 1) // num_rows  # Number of columns in the grid
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(4 * num_cols, 3 * num_rows), sharex=True)

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Enumerate through columns to plot and plot on subplots
for i, col in enumerate(plot_cols):
    axes[i].plot(df.index, df[col])
    axes[i].set_title(col)  # Add title to each subplot

# Hide x-axis label on all subplots except the last row
for ax in axes[:-num_cols]:
    ax.set_xlabel('')

# Common x-axis label for the bottom row of subplots
for ax in axes[-num_cols:]:
    ax.set_xlabel('Time')

# Adjust layout (optional)
plt.tight_layout()

# Remove any unused subplots
for i in range(len(plot_cols), len(axes)):
    fig.delaxes(axes[i])

plt.show()